In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

from tqdm import tqdm
import ast

from multiprocessing import Pool

In [2]:
secuencias_entrenamiento_df = pd.read_csv('../Data/secuencias_entrenamiento.csv')
secuencias_entrenamiento_df

,session_id,item_id_salida,secuencia_items_ids,secuencia_fechas
0,3,15085,"[9655, 9655]","['2020-12-18 21:19:48.093', '2020-12-18 21:25:..."
1,13,18626,[15654],['2020-03-13 19:35:27.136']
2,18,24911,"[4026, 2507, 18316]","['2020-08-26 19:15:47.232', '2020-08-26 19:16:..."
3,19,12534,"[19896, 27937, 12804, 25772, 6341, 23687, 8316...","['2020-11-02 16:30:36.378', '2020-11-02 16:30:..."
4,24,13226,"[2927, 2927, 16064, 11662, 434, 18539, 10414, ...","['2020-02-26 17:22:48.903', '2020-02-26 17:23:..."
...,...,...,...,...
999995,4439986,2915,"[23502, 13914, 13403, 19310, 12373, 27733]","['2021-05-13 11:46:56.606', '2021-05-13 11:48:..."
999996,4439990,8786,"[22093, 17429, 1389, 1131, 4062, 26011, 4028, ...","['2020-08-22 11:38:48.785', '2020-08-22 11:39:..."
999997,4439994,21630,[25357],['2020-11-27 20:08:37.262']
999998,4439999,16962,"[6007, 8502, 13056, 2173, 8433, 1299, 15853]","['2020-11-27 10:52:12.577', '2020-11-27 10:53:..."


In [5]:
encoded_items_df = pd.read_csv('../Data/items_item2vec_encoded.csv')
encoded_items_df

,item_id,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
0,2,0.000000e+00,0.000000e+00,8.259485e-38,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,...,1.147871e-04,1.196613e-25,0.000000,0.000000e+00,4.975298e-21,0.000000,1.000000e+00,1.283712e-37,0.000000e+00,5.708601e-11
1,3,7.226150e-16,2.417073e-01,2.325430e-14,3.434116e-10,9.999780e-01,2.084914e-19,2.359839e-04,7.360519e-18,3.367935e-12,...,6.631301e-28,1.442925e-15,1.000000,7.131817e-04,9.999999e-01,0.034609,8.714515e-01,5.728165e-07,1.033759e-04,9.978011e-01
2,4,7.434464e-30,7.750213e-18,0.000000e+00,1.445317e-29,1.000000e+00,0.000000e+00,0.000000e+00,3.267208e-25,9.452282e-33,...,0.000000e+00,5.438013e-28,1.000000,5.429324e-03,1.000000e+00,1.000000,9.924654e-01,6.889178e-05,9.211239e-23,2.146675e-14
3,7,5.490076e-37,0.000000e+00,1.191505e-21,9.322182e-12,4.260253e-13,5.699080e-01,1.000000e+00,4.086731e-30,5.050279e-12,...,1.000000e+00,0.000000e+00,0.000000,1.472102e-12,4.147344e-34,0.000000,0.000000e+00,1.565254e-36,1.000000e+00,1.000000e+00
4,8,2.423661e-25,2.795917e-11,6.869838e-35,1.957358e-24,1.000000e+00,1.214310e-37,0.000000e+00,2.739703e-17,1.734631e-19,...,0.000000e+00,1.350020e-19,1.000000,7.157927e-10,1.000000e+00,1.000000,2.604076e-02,1.155714e-02,3.667003e-09,1.811880e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23686,28139,9.726211e-01,2.284531e-11,2.776817e-11,5.597994e-02,1.026659e-02,2.461190e-02,1.000000e+00,3.444716e-18,1.398369e-05,...,4.171538e-03,1.981965e-04,0.961271,7.362516e-17,3.345782e-01,0.000326,3.236383e-25,6.360826e-16,9.999509e-01,9.999608e-01
23687,28140,9.997767e-01,2.538743e-08,7.991548e-11,2.896282e-21,3.695540e-04,9.091385e-18,2.061393e-09,1.558076e-14,5.036157e-05,...,3.120203e-29,1.000000e+00,1.000000,2.877177e-09,2.796884e-04,0.615862,9.811885e-01,1.042302e-14,2.447064e-09,9.952277e-01
23688,28141,2.941844e-15,7.376524e-11,3.965195e-17,8.104620e-05,9.995428e-01,5.364074e-11,4.411794e-07,4.666649e-08,1.239170e-11,...,2.487879e-37,3.125461e-19,1.000000,4.307819e-08,1.000000e+00,0.987323,9.997515e-01,3.231751e-02,1.217761e-02,8.776583e-06
23689,28142,0.000000e+00,1.112993e-36,1.000000e+00,1.000000e+00,3.748123e-22,1.000000e+00,1.000000e+00,1.000000e+00,1.470198e-26,...,1.000000e+00,0.000000e+00,0.000000,5.781284e-18,0.000000e+00,0.000000,6.874122e-17,0.000000e+00,1.000000e+00,5.961677e-13


In [6]:
def normalize_len_sequence(sequence_list, len_output = 10):
    
    if len(sequence_list) >= len_output:
        
        output_list = sequence_list[-1*len_output:]
        
    else:
        
        diference_len = len_output - len(sequence_list)
        padding = [0] * diference_len 
        output_list = padding + sequence_list
        
    return output_list


def create_tensors_for_rnn(secuencias_df, items_df, length_sequence = 10):
    
    X = []
    Y = []
    
    example_tensor = items_df.loc[[0]].drop('item_id', axis=1).to_numpy()
    
    for index, row in tqdm(secuencias_df.iterrows()):
        
        secuencia_items = ast.literal_eval(row.secuencia_items_ids)
        secuencia_items_normalizada = normalize_len_sequence(secuencia_items, length_sequence)
        
        secuencia_fechas_items = ast.literal_eval(row.secuencia_fechas)
        secuencia_fechas_items_normalizada = normalize_len_sequence(secuencia_fechas_items, length_sequence)
        
        X_secuencia_individual = []
        
        for item in secuencia_items_normalizada:
                        
            if item == 0:
                
                item_data_tensor = np.zeros(example_tensor.shape)
                X_secuencia_individual.append(item_data_tensor)
                
            else:
                
                item_data_tensor = items_df.loc[items_df.item_id == item].drop('item_id', axis=1).to_numpy()
                X_secuencia_individual.append(item_data_tensor)
        
        
        X_secuencia_individual = np.array(X_secuencia_individual)
        X_secuencia_individual = np.reshape(X_secuencia_individual, (X_secuencia_individual.shape[0], X_secuencia_individual.shape[2]))
        
        X.append(X_secuencia_individual)
        Y.append(items_df.loc[items_df.item_id == row.item_id_salida].drop('item_id', axis=1).to_numpy())
        

    X = np.array(X)
    Y = np.array(Y)
    Y = np.reshape(Y, (Y.shape[0], Y.shape[2]))
    
    return X, Y

# Creacion de un dataset con secuencias de una longitud de 10 sesiones con los valores de los items en espacio latente del autoencoder

In [7]:
X_10,Y_10 = create_tensors_for_rnn(secuencias_entrenamiento_df, encoded_items_df, 10)

1000000it [29:49, 558.77it/s]


In [8]:
X_10.shape

(1000000, 10, 64)

In [9]:
Y_10.shape

(1000000, 64)

In [10]:
np.save('../Data/X_10', X_10)
np.save('../Data/Y_10', Y_10)